In [1]:
from keras.models import Sequential
from keras.layers import Dense
import numpy as np
import sys, os, string

characters = string.printable
char_indices = dict((c, i) for i, c in enumerate(characters))
indices_char = dict((i, c) for i, c in enumerate(characters))

INPUT_VOCAB_SIZE = len(characters)

def encode_one_hot(line):
    x = np.zeros((len(line), INPUT_VOCAB_SIZE))
    for i, c in enumerate(line):
        if c in characters:
            index = char_indices[c]
        else:
            index = char_indices[' ']
        x[i][index] = 1 
    return x

def decode_one_hot(x):
    s = []
    for onehot in x:
        one_index = np.argmax(onehot) 
        s.append(indices_char[one_index]) 
    return ''.join(s)
    
def normalization_layer_set_weights(n_layer):
    wb = []
    w = np.zeros((INPUT_VOCAB_SIZE, INPUT_VOCAB_SIZE), dtype=np.float32)
    b = np.zeros((INPUT_VOCAB_SIZE), dtype=np.float32)
    # Let lower case letters go through
    for c in string.ascii_lowercase:
        i = char_indices[c]
        w[i, i] = 1
    # Map capitals to lower case
    for c in string.ascii_uppercase:
        i = char_indices[c]
        il = char_indices[c.lower()]
        w[i, il] = 1
    # Map all non-letters to space
    sp_idx = char_indices[' ']
    for c in [c for c in list(string.printable) if c not in list(string.ascii_letters)]:
        i = char_indices[c]
        w[i, sp_idx] = 1

    wb.append(w)
    wb.append(b)
    n_layer.set_weights(wb)
    return n_layer

def convert_to_leet(n_layer):
    l_wb = []
    l_w = np.zeros((INPUT_VOCAB_SIZE, INPUT_VOCAB_SIZE), dtype=np.float32)
    l_b = np.zeros((INPUT_VOCAB_SIZE), dtype=np.float32)
    leet_map = {'a':'4', 'b':'8', 'c':'<', 'd':')', 'e':'3', 'f':'f', 'g':'6', 
                'h':'#', 'i':'|', 'j':']', 'k':'k', 'l':'1', 'm':'m', 'n':'^',
                'o':'0', 'p':'9', 'q':'&', 'r':'r', 's':'5', 't':'+', 'u':'u',
                'v':'v', 'w':'w', 'x':'x', 'y':'y', 'z':'7'}
    # Mapping letters to their leet counterparts
    for c in characters:
        i = char_indices[c]
        if c in leet_map:
            i1 = char_indices[leet_map[c]]
        else:
            i1 = char_indices[c]
                
        l_w[i, i1] = 1
        
    l_wb.append(l_w)
    l_wb.append(l_b)
    n_layer.set_weights(l_wb)
    return n_layer


def build_model():
    # Normalize characters using a dense layer
    model = Sequential()
    dense_layer_1 = Dense(INPUT_VOCAB_SIZE, 
                        input_shape=(INPUT_VOCAB_SIZE,),
                        activation='softmax')
    dense_layer_2 = Dense(INPUT_VOCAB_SIZE, 
                        input_shape=(INPUT_VOCAB_SIZE,),
                        activation='softmax')
    model.add(dense_layer_1)
    model.add(dense_layer_2)
    return model

model = build_model()
model.summary()
normalization_layer_set_weights(model.layers[0])

# normalized_model = normalization_layer_set_weights(model.layers[0])
#final_model = 
convert_to_leet(model.layers[1])

count = 0
fp = "../../pride-and-prejudice.txt"
#sys.argv[1]
with open(fp) as f:
    for line in f:
        if line.isspace(): continue
        batch = encode_one_hot(line)
        preds = model.predict(batch)
        #preds = final_model.predict(batch)
        normal = decode_one_hot(preds)
        print(normal)
        count += 1
print("Count:", count)

2022-05-29 15:56:20.751673: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-05-29 15:56:20.752007: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Metal device set to: Apple M1 Pro

systemMemory: 32.00 GB
maxCacheSize: 10.67 GB

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 100)               10100     
                                                                 
 dense_1 (Dense)             (None, 100)               10100     
                                                                 
Total params: 20,200
Trainable params: 20,200
Non-trainable params: 0
_________________________________________________________________


FileNotFoundError: [Errno 2] No such file or directory: '../pride-and-prejudice.txt'